In [3]:
import librosa
import numpy as np

def extract_features(file_path, sr=16000, n_mfcc=13):
    y, _ = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)


# Prepare Training Data

In [7]:
import pandas as pd
import os

train_df = pd.read_csv(r"C:\Users\ayush\Downloads\dataset\train.csv")
X = []
y = []

for idx, row in train_df.iterrows():
    file_path = os.path.join('audios_train', row['filename'])
    features = extract_features(file_path)
    X.append(features)
    y.append(row['label'])

X = np.array(X)
y = np.array(y)


c:\Users\ayush\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\ayush\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\ayush\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


# 4. Model Training

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluating the model
print("MSE:", mean_squared_error(y_val, y_pred))
print("R² Score:", r2_score(y_val, y_pred))

# Predict and evaluate
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))


MSE: 1.3146067415730338
R² Score: 0.12436932391523725
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.52      0.39      0.45        28
           3       0.30      0.56      0.39        18
           4       0.32      0.35      0.33        17
           5       0.62      0.40      0.49        25

    accuracy                           0.42        89
   macro avg       0.35      0.34      0.33        89
weighted avg       0.46      0.42      0.42        89



c:\Users\ayush\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ayush\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ayush\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Inference on Test Set

In [25]:
test_features = []

# Loop through each file in the test dataframe
for file in test_df['filename']:
    file_path = os.path.join('audios_test', file)
    features = extract_features(file_path)
    test_features.append(features)

# Convert list of features to a NumPy array
test_features = np.array(test_features)

# Make predictions
predictions = clf.predict(test_features)

In [26]:
predictions

array([2, 2, 4, 3, 2, 2, 2, 3, 4, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 5, 3, 2,
       2, 4, 3, 5, 3, 3, 3, 3, 2, 2, 2, 5, 4, 2, 3, 3, 2, 2, 5, 2, 2, 4,
       3, 3, 2, 2, 3, 4, 2, 4, 4, 4, 3, 3, 5, 3, 2, 3, 4, 4, 5, 3, 3, 3,
       3, 2, 4, 5, 3, 4, 3, 3, 3, 2, 4, 4, 3, 4, 5, 3, 2, 2, 3, 3, 3, 4,
       4, 2, 3, 5, 2, 4, 2, 3, 3, 5, 4, 4, 5, 4, 3, 5, 4, 5, 5, 3, 5, 3,
       2, 4, 3, 3, 4, 4, 5, 5, 5, 3, 4, 3, 5, 5, 4, 4, 4, 3, 4, 3, 4, 5,
       3, 3, 5, 3, 3, 3, 3, 4, 5, 4, 3, 3, 3, 2, 2, 5, 5, 5, 4, 4, 5, 3,
       4, 4, 4, 4, 3, 5, 4, 5, 3, 4, 3, 5, 5, 3, 4, 5, 5, 4, 4, 5, 3, 4,
       4, 4, 5, 5, 3, 4, 3, 4, 3, 3, 4, 5, 3, 5, 4, 4, 4, 5, 5])

# Create Submission File

In [30]:
submission = pd.DataFrame({
    'file_name': test_df['filename'],
    'label': predictions
})
submission.to_csv('submit.csv', index=False)


In [32]:
df = pd.read_csv('submit.csv')
print(df.head())

        file_name  label
0   audio_706.wav      2
1   audio_800.wav      2
2    audio_68.wav      4
3  audio_1267.wav      3
4   audio_683.wav      2
